<a href="https://colab.research.google.com/github/Tsung-Hung/dummy-git/blob/master/%E6%AD%A1%E8%BF%8E%E4%BD%BF%E7%94%A8_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch 
import torch.nn as nn

# LayerNorm replace by BatchNorm

In [4]:
batch, sentence_length, embedding_dim = 2, 3, 5
embedding = torch.randn(batch, sentence_length, embedding_dim)
layer_norm = nn.LayerNorm(embedding_dim)
# Activate module
origin = layer_norm(embedding)
origin

tensor([[[ 0.2980, -1.9365,  0.1729,  0.5633,  0.9022],
         [ 1.2849, -1.1374,  1.0650, -0.9111, -0.3014],
         [ 1.5007,  0.6492, -0.6085, -1.3905, -0.1509]],

        [[-1.7504, -0.1274,  1.0473, -0.0738,  0.9042],
         [ 0.6699,  0.6456, -0.8121,  1.0422, -1.5455],
         [ 0.1314,  0.5984,  1.1941, -0.1408, -1.7830]]],
       grad_fn=<NativeLayerNormBackward0>)

In [5]:
def Layer_norm(x):
    '''
    BatchNorm2d
    '''
    batch, sentence_length, embedding_dim = x.shape
    x = x.reshape(1, batch*sentence_length, embedding_dim, 1)
    layer_norm = nn.BatchNorm2d(x.shape[1], track_running_stats=False, affine=True)
    output = layer_norm(x)
    output = output.reshape(output.shape[1], output.shape[2]).reshape(batch, sentence_length, embedding_dim)
    return output

test = Layer_norm(embedding)
test

tensor([[[ 0.2980, -1.9365,  0.1729,  0.5633,  0.9022],
         [ 1.2849, -1.1374,  1.0650, -0.9111, -0.3014],
         [ 1.5007,  0.6492, -0.6085, -1.3905, -0.1509]],

        [[-1.7504, -0.1274,  1.0473, -0.0738,  0.9042],
         [ 0.6699,  0.6456, -0.8121,  1.0422, -1.5455],
         [ 0.1314,  0.5984,  1.1941, -0.1408, -1.7830]]],
       grad_fn=<ReshapeAliasBackward0>)

In [6]:
def Layer_norm2(x):
    '''
    BatchNorm1d
    '''
    batch, sentence_length, embedding_dim = x.shape
    x = x.reshape(1, batch*sentence_length, embedding_dim)
    layer_norm = nn.BatchNorm1d(x.shape[1], track_running_stats=False, affine=True)
    output = layer_norm(x)
    output = output.reshape(output.shape[1], output.shape[2]).reshape(batch, sentence_length, embedding_dim)
    return output

test = Layer_norm2(embedding)
test

tensor([[[ 0.2980, -1.9365,  0.1729,  0.5633,  0.9022],
         [ 1.2849, -1.1374,  1.0650, -0.9111, -0.3014],
         [ 1.5007,  0.6492, -0.6085, -1.3905, -0.1509]],

        [[-1.7504, -0.1274,  1.0473, -0.0738,  0.9042],
         [ 0.6699,  0.6456, -0.8121,  1.0422, -1.5455],
         [ 0.1314,  0.5984,  1.1941, -0.1408, -1.7830]]],
       grad_fn=<ReshapeAliasBackward0>)

# matmul 4D tensor

In [7]:
a1 = torch.rand(2,2,4,8)
a2 = torch.rand(2,2,6,8)

result = torch.matmul(a1, a2.transpose(-2,-1))
print('Shape : ', result.shape)
result

Shape :  torch.Size([2, 2, 4, 6])


tensor([[[[2.8504, 3.7758, 3.2092, 3.7202, 1.8101, 2.6187],
          [1.7503, 2.0641, 2.3779, 2.3639, 1.9432, 1.8165],
          [0.8607, 1.0376, 0.8389, 1.2107, 0.6391, 0.7826],
          [1.1995, 1.9408, 1.5099, 1.9302, 1.4166, 1.3188]],

         [[1.8287, 2.6204, 2.0797, 3.0608, 2.4285, 1.3165],
          [1.6778, 2.1464, 1.4736, 2.2671, 2.4186, 1.0302],
          [1.2585, 2.2695, 1.2953, 2.4700, 1.9442, 1.3790],
          [1.0399, 1.5724, 0.7590, 1.4151, 1.7951, 0.9365]]],


        [[[1.5949, 1.9834, 1.7013, 1.5605, 1.2784, 1.8137],
          [1.5951, 2.2026, 1.6770, 1.0176, 1.2802, 2.4490],
          [1.7381, 1.7665, 1.6077, 1.3219, 1.4200, 1.3843],
          [1.7357, 2.1351, 1.7021, 1.3528, 1.6711, 1.7599]],

         [[1.4509, 1.1037, 0.6165, 1.1103, 1.3576, 1.1042],
          [3.2681, 3.2637, 1.4212, 3.2439, 3.2147, 2.5006],
          [2.5874, 2.4536, 1.1247, 2.7364, 2.8152, 2.1790],
          [1.5235, 1.4493, 0.6476, 1.2378, 1.5088, 1.3064]]]])

In [8]:
def new_matmul1(a1,a2):

    k = a1.shape[0]*a1.shape[1]    
    mul_list = []
    for i in range(a1.shape[0]):
        for j in range(a1.shape[1]):
            
            mul = torch.matmul(a1[i,j,:],a2[i,j,:])                
            mul_list.append(mul)

    combine = torch.stack(mul_list)  
    combine = combine.view(a1.shape[0],a1.shape[1],combine.shape[1],combine.shape[2])
    
    return combine

result1 = new_matmul1(a1, a2.transpose(-2,-1))
print('Shape : ', result1.shape)
print(result1)

Shape :  torch.Size([2, 2, 4, 6])
tensor([[[[2.8504, 3.7758, 3.2092, 3.7202, 1.8101, 2.6187],
          [1.7503, 2.0641, 2.3779, 2.3639, 1.9432, 1.8165],
          [0.8607, 1.0376, 0.8389, 1.2107, 0.6391, 0.7826],
          [1.1995, 1.9408, 1.5099, 1.9302, 1.4166, 1.3188]],

         [[1.8287, 2.6204, 2.0797, 3.0608, 2.4285, 1.3165],
          [1.6778, 2.1464, 1.4736, 2.2671, 2.4186, 1.0302],
          [1.2585, 2.2695, 1.2953, 2.4700, 1.9442, 1.3790],
          [1.0399, 1.5724, 0.7590, 1.4151, 1.7951, 0.9365]]],


        [[[1.5949, 1.9834, 1.7013, 1.5605, 1.2784, 1.8137],
          [1.5951, 2.2026, 1.6770, 1.0176, 1.2802, 2.4490],
          [1.7381, 1.7665, 1.6077, 1.3219, 1.4200, 1.3843],
          [1.7357, 2.1351, 1.7021, 1.3528, 1.6711, 1.7599]],

         [[1.4509, 1.1037, 0.6165, 1.1103, 1.3576, 1.1042],
          [3.2681, 3.2637, 1.4212, 3.2439, 3.2147, 2.5006],
          [2.5874, 2.4536, 1.1247, 2.7364, 2.8152, 2.1790],
          [1.5235, 1.4493, 0.6476, 1.2378, 1.5088, 1.3064]

In [9]:
def new_matmul2(a1, a2):

    combine = torch.stack([torch.matmul(a1[i][j], a2[i][j]) for i in range(a1.shape[0]) for j in range(a1.shape[1])], dim=0)        
    return combine.reshape(a1.shape[0], a1.shape[1], a1.shape[2], a2.shape[3])

result2 = new_matmul1(a1, a2.transpose(-2,-1))
print('Shape : ', result2.shape)
print(result2)
print(torch.all(result==result2))

Shape :  torch.Size([2, 2, 4, 6])
tensor([[[[2.8504, 3.7758, 3.2092, 3.7202, 1.8101, 2.6187],
          [1.7503, 2.0641, 2.3779, 2.3639, 1.9432, 1.8165],
          [0.8607, 1.0376, 0.8389, 1.2107, 0.6391, 0.7826],
          [1.1995, 1.9408, 1.5099, 1.9302, 1.4166, 1.3188]],

         [[1.8287, 2.6204, 2.0797, 3.0608, 2.4285, 1.3165],
          [1.6778, 2.1464, 1.4736, 2.2671, 2.4186, 1.0302],
          [1.2585, 2.2695, 1.2953, 2.4700, 1.9442, 1.3790],
          [1.0399, 1.5724, 0.7590, 1.4151, 1.7951, 0.9365]]],


        [[[1.5949, 1.9834, 1.7013, 1.5605, 1.2784, 1.8137],
          [1.5951, 2.2026, 1.6770, 1.0176, 1.2802, 2.4490],
          [1.7381, 1.7665, 1.6077, 1.3219, 1.4200, 1.3843],
          [1.7357, 2.1351, 1.7021, 1.3528, 1.6711, 1.7599]],

         [[1.4509, 1.1037, 0.6165, 1.1103, 1.3576, 1.1042],
          [3.2681, 3.2637, 1.4212, 3.2439, 3.2147, 2.5006],
          [2.5874, 2.4536, 1.1247, 2.7364, 2.8152, 2.1790],
          [1.5235, 1.4493, 0.6476, 1.2378, 1.5088, 1.3064]

In [12]:
def new_matmul3(a1, a2):
  '''
  a1 = (2,2,4,8)
  a2 = (2,2,6,8)
  torch(a2, a1.T)
  '''
  B1, N1, H1, T1 = a1.shape
  B2, N2, T2, H2 = a2.shape
  a1 = a1.reshape(B1 * N1 * H1, T1)
  #print('a1 : ', a1.shape)
  a2 = a2.permute(2,0,1,3)
  a2 = a2.reshape(T2, B2 * N2 * H2)
  #print('a2 : ', a2.shape)
  result = torch.matmul(a1, a2)
  result = torch.stack([result[i*H1:(i+1)*H1 ,i*H2:(i+1)*H2] for i in range(B1*N1)])
  result = result.reshape(B1,N1,H1,H2)
  return result

result3 = new_matmul3(a1, a2.transpose(-2,-1))
print('Shape : ', result3.shape)
print(result3)

Shape :  torch.Size([2, 2, 4, 6])
tensor([[[[2.8504, 3.7758, 3.2092, 3.7202, 1.8101, 2.6187],
          [1.7503, 2.0641, 2.3779, 2.3639, 1.9432, 1.8165],
          [0.8607, 1.0376, 0.8389, 1.2107, 0.6391, 0.7826],
          [1.1995, 1.9408, 1.5099, 1.9302, 1.4166, 1.3188]],

         [[1.8287, 2.6204, 2.0797, 3.0608, 2.4285, 1.3165],
          [1.6778, 2.1464, 1.4736, 2.2671, 2.4186, 1.0302],
          [1.2585, 2.2695, 1.2953, 2.4700, 1.9442, 1.3790],
          [1.0399, 1.5724, 0.7590, 1.4151, 1.7951, 0.9365]]],


        [[[1.5949, 1.9834, 1.7013, 1.5605, 1.2784, 1.8137],
          [1.5951, 2.2026, 1.6770, 1.0176, 1.2802, 2.4490],
          [1.7381, 1.7665, 1.6077, 1.3219, 1.4200, 1.3843],
          [1.7357, 2.1351, 1.7021, 1.3528, 1.6711, 1.7599]],

         [[1.4509, 1.1037, 0.6165, 1.1103, 1.3576, 1.1042],
          [3.2681, 3.2637, 1.4212, 3.2439, 3.2147, 2.5006],
          [2.5874, 2.4536, 1.1247, 2.7364, 2.8152, 2.1790],
          [1.5235, 1.4493, 0.6476, 1.2378, 1.5088, 1.3064]

In [20]:
def new_matmul3_plus(a1, a2):
    '''
    a1 = (2,2,4,8)
    a2 = (2,2,6,8)
    torch(a2, a1.T)
    '''
    B1, N1, H1, T1 = a1.shape
    B2, N2, T2, H2 = a2.shape
    #print(H1,T1,T2,H2)
    a1 = a1.reshape(B1 * N1 * H1, T1)

    a2 = a2.permute(2,0,1,3)
    a2 = a2.reshape(T2, B2 * N2 * H2)

    result = torch.matmul(a1, a2)
    result = result.reshape(B1*N1, B1*N1, H1, H2).transpose(1,2)
    result = result.reshape(B1*N1*B1*N1, H1, H2)

    index = torch.arange(0, B1*N1*B1*N1, (B1*N1)+1)
    result = result[index]
    print(result.shape)
    #result = result.reshape(1, N1, H1, H2)
    result = result.unsqueeze(0)
    print(result.shape)

    return result


a1 = torch.rand(1,4,150,16)
a2 = torch.rand(1,4,10,16)
result3_plus = new_matmul3_plus(a1, a2.transpose(-2,-1))
print('Shape : ', result3_plus.shape)
print(result3_plus)   

torch.Size([4, 150, 10])
torch.Size([1, 4, 150, 10])
Shape :  torch.Size([1, 4, 150, 10])
tensor([[[[3.4355, 4.9947, 3.1470,  ..., 3.9659, 4.3456, 4.3777],
          [4.5843, 4.7847, 4.7374,  ..., 5.3907, 5.4319, 5.3982],
          [4.1750, 5.1875, 3.3297,  ..., 4.0148, 4.9876, 5.5538],
          ...,
          [3.9169, 4.2030, 4.3425,  ..., 4.8236, 4.9344, 6.0989],
          [5.9292, 4.9234, 6.1471,  ..., 5.7745, 4.8960, 4.2990],
          [4.6000, 4.5656, 5.8975,  ..., 5.0430, 2.5960, 6.2320]],

         [[5.0545, 4.1158, 4.3191,  ..., 4.5189, 4.4188, 4.0376],
          [3.5234, 4.4329, 4.4406,  ..., 4.7067, 1.6115, 5.1223],
          [2.9669, 3.2210, 3.2740,  ..., 3.6212, 3.8104, 4.0797],
          ...,
          [5.5760, 6.2677, 3.9996,  ..., 6.0426, 5.8895, 6.1199],
          [3.9105, 4.4843, 4.0873,  ..., 5.3853, 5.5693, 6.0275],
          [3.5881, 4.5598, 3.2345,  ..., 3.6201, 3.6136, 4.9670]],

         [[2.8178, 3.1639, 4.0840,  ..., 2.8696, 1.0982, 2.5587],
          [4.4225,

In [11]:
def new_matmul4(a1, a2):
  '''
  a1 = (2,2,4,8)  
  a2 = (2,2,6,8)
  a1_matrix = (16,32)
  a2_matrix = (32,24)
  '''
  a1_matrix = torch.block_diag(*[a1[i][j] for i in range(a1.shape[0]) for j in range(a1.shape[1])])
  a2_matrix = torch.block_diag(*[a2[i][j] for i in range(a2.shape[0]) for j in range(a2.shape[1])])
  identity = torch.cat([torch.eye(a2.shape[3]) for _ in range(a2.shape[0]*a2.shape[1])]).cuda()
  a = torch.ones(a2.shape[0]*a2.shape[1],a2.shape[3])
  identity2 = torch.diag_embed(a).reshape(a2.shape[0]*a2.shape[1]*a2.shape[3], a2.shape[3])
  result = torch.matmul(a1_matrix, a2_matrix)
  result = torch.matmul(result, identity2)
  return result

result4 = new_matmul4(a1,a2.transpose(-2,-1))
print('Shape : ', result4.shape)
print(result4)

NameError: ignored

In [2]:
def new_matmul_final(a1, a2):
  '''
  a1 = (2,2,4,8)  
  a2 = (2,2,6,8)
  a1_matrix = (16,32)
  a2_matrix = (32,24)
  '''
  a1_matrix = torch.block_diag(*[a1[i][j] for i in range(a1.shape[0]) for j in range(a1.shape[1])])
  a2_matrix = torch.block_diag(*[a2[i][j] for i in range(a2.shape[0]) for j in range(a2.shape[1])])
  #identity = torch.cat([torch.eye(a2.shape[3]) for _ in range(a2.shape[0]*a2.shape[1])]).cuda()
  a = torch.ones(a2.shape[0]*a2.shape[1],a2.shape[3])
  #identity2 = torch.diag_embed(a).reshape(a2.shape[0]*a2.shape[1]*a2.shape[3], a2.shape[3]).cuda()
  result = torch.matmul(a1_matrix, a2_matrix)
  #result = torch.matmul(result, identity2)
  return result

#result4 = new_matmul4(a1,a2.transpose(-2,-1))
#print('Shape : ', result4.shape)
#print(result4)

In [ ]:
a2 = torch.rand(1,2,3,4)
a2

tensor([[[[0.7210, 0.2294, 0.4558, 0.0930],
          [0.2017, 0.0607, 0.4425, 0.3461],
          [0.5573, 0.3538, 0.4102, 0.7203]],

         [[0.8404, 0.7805, 0.1481, 0.0404],
          [0.7765, 0.7399, 0.3273, 0.6756],
          [0.1364, 0.3438, 0.1867, 0.5070]]]])

In [ ]:

B2, N2, T2, H2 = a2.shape
a2 = a2.reshape( B2 * N2* T2, H2)
print(a2.shape)
one = torch.ones(B2*N2, H2)
a2_I = torch.diag_embed(one).permute(1,0,2).reshape(H2, B2*N2*H2)
a2_m = torch.matmul(a2, a2_I)
a2_m

torch.Size([6, 4])


tensor([[0.7210, 0.2294, 0.4558, 0.0930, 0.7210, 0.2294, 0.4558, 0.0930],
        [0.2017, 0.0607, 0.4425, 0.3461, 0.2017, 0.0607, 0.4425, 0.3461],
        [0.5573, 0.3538, 0.4102, 0.7203, 0.5573, 0.3538, 0.4102, 0.7203],
        [0.8404, 0.7805, 0.1481, 0.0404, 0.8404, 0.7805, 0.1481, 0.0404],
        [0.7765, 0.7399, 0.3273, 0.6756, 0.7765, 0.7399, 0.3273, 0.6756],
        [0.1364, 0.3438, 0.1867, 0.5070, 0.1364, 0.3438, 0.1867, 0.5070]])

In [ ]:
one = torch.ones(B2*N2, H2)
a2_I = torch.diag_embed(one).permute(1,0,2).reshape(H2, B2*N2*H2)
a2_I

tensor([[1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 0., 1.]])

In [ ]:
#test = torch.eye(a2.shape[1])
one = torch.ones(2,4)
a2_I = torch.diag_embed(one)
a2_I
#test = torch.matmul(a2,test)
#test

tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]],

        [[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]]])

In [ ]:
a1 = torch.rand(1,4,15000,16).cuda()
a2 = torch.rand(1,4,1000,16).cuda()

%timeit -n 100 -r 7 torch.matmul(a1, a2.transpose(-2,-1))

25.4 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit -n 100 -r 7 new_matmul1(a1, a2.transpose(-2,-1))

3.99 ms ± 1.46 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit -n 100 -r 7 new_matmul2(a1, a2.transpose(-2,-1))

3.57 ms ± 46.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit -n 200 -r 7 new_matmul3(a1, a2.transpose(-2,-1))

The slowest run took 8.39 times longer than the fastest. This could mean that an intermediate result is being cached.
2.01 ms ± 2.06 ms per loop (mean ± std. dev. of 7 runs, 200 loops each)


In [ ]:
%timeit -n 100 -r 7 new_matmul4(a1, a2.transpose(-2,-1))

11.5 ms ± 7.34 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
a1_matrix = torch.block_diag(*[a1[i][j] for i in range(a1.shape[0]) for j in range(a1.shape[1])])
a2_matrix = torch.block_diag(*[a2[i][j] for i in range(a2.shape[0]) for j in range(a2.shape[1])])

%timeit -n 100 -r 7 torch.matmul(a1_matrix, a2_matrix.transpose(-2,-1))

746 µs ± 36.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
from numpy import matrix
A = matrix([[1,0,0],[0,1,0],[0,0,1],[1,0,0],[0,1,0],[0,0,1]])
A2 = A.I
B = matrix([[1,3,2],[6,1,2],[3,1,1],[1,4,1],[1,1,2],[3,2,1]])
B * A2 * 2

matrix([[1., 3., 2., 1., 3., 2.],
        [6., 1., 2., 6., 1., 2.],
        [3., 1., 1., 3., 1., 1.],
        [1., 4., 1., 1., 4., 1.],
        [1., 1., 2., 1., 1., 2.],
        [3., 2., 1., 3., 2., 1.]])

In [ ]:
AB = matrix([[1,3,2,0,0,0],[6,1,2,0,0,0],[3,1,1,0,0,0],[0,0,0,1,4,1],[0,0,0,1,1,2],[0,0,0,3,2,1]])
I1 = matrix([[1,0,0],[0,1,0],[0,0,1],[1,0,0],[0,1,0],[0,0,1]])
I2 = matrix([[1,0,0,1,0,0],[0,1,0,0,1,0],[0,0,1,0,0,1]])
res = AB*I1
res

matrix([[1, 3, 2],
        [6, 1, 2],
        [3, 1, 1],
        [1, 4, 1],
        [1, 1, 2],
        [3, 2, 1]])